the purpose of this notebook is to take the big dataframe created in 07.01-baseline_data_merging, and make an aggregate we can use for our baseline "worst case" scenario model. This will then be fed into R, where we will use the MICE package to impute data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/04042019_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals, 14478 icustay_id
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 93.8 ms


In [3]:
#pd.to_datetime(final_pt_df2['t_0'])- pd.to_datetime(final_pt_df2['t_0'])
#final_pt_df2
(pd.to_datetime(final_pt_df2['t_0'])-pd.to_datetime(final_pt_df2['t_0_sc'])).describe() #ssc time - ab time
#(pd.to_datetime(final_pt_df2['ICU_admit'])-pd.to_datetime(final_pt_df2['t_0'])).describe()

count                     14478
mean     0 days 01:55:28.064649
std      0 days 10:00:09.874180
min           -1 days +00:00:00
25%           -1 days +19:37:00
50%             0 days 02:45:00
75%             0 days 08:39:00
max             1 days 00:00:00
dtype: object

time: 22.2 ms


In [4]:
#final_pt_df2
(pd.to_datetime(final_pt_df2['t_0'])-pd.to_datetime(final_pt_df2['ICU_admit'])).describe() #icu admit time - ab time

count                     14478
mean     0 days 06:52:07.033775
std      0 days 17:39:21.503063
min           -1 days +12:00:21
25%           -1 days +21:47:17
50%      0 days 02:56:52.500000
75%      0 days 08:05:23.250000
max             3 days 23:46:07
dtype: object

time: 17.3 ms


In [5]:
(pd.to_datetime(final_pt_df2['t_0_sc'])-pd.to_datetime(final_pt_df2['ICU_admit'])).describe() #icu admit time - ssc time

count                       14478
mean       0 days 04:56:38.969125
std        0 days 20:01:44.531744
min             -2 days +12:08:23
25%      -1 days +20:15:15.500000
50%             -1 days +23:05:58
75%        0 days 05:21:32.250000
max               4 days 21:07:19
dtype: object

time: 26.3 ms


In [6]:
#importing in all clinical_variable files
date= '04042019'
os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/')
big_df= pd.read_csv(Path(wd+'/data/processed/merged/%s_longdf_preImp.csv' %(date),  index_col=0))
big_df= big_df.reset_index(drop=True).iloc[:,1:]

time: 12 s


In [7]:
#removing firstpos else neg ssc col
big_df=big_df.loc[:,list(big_df.columns!="first_pos_else_neg_ssc")]

time: 481 ms


In [8]:
big_df.groupby('label')['value'].describe().sort_values('unique')
##categorical variables:
#rrt, phenylephrine, norepinephrine, vasopressin, dobutamine, dopamine, epinephrine, leukocyte, vent_recieved


,count,unique,top,freq
label,,,,
leukocyte,14478,2,Neg/Not_tested,12278
norepinephrine,34427,2,1.0,22788
o2_flow,14734,2,0.0,13734
gender,14478,2,M,7940
phenylephrine,26635,2,1.0,14378
epinephrine,16080,2,0.0,14188
dopamine,19236,2,0.0,13623
nitrite,14478,2,Neg/Not_tested,14061
dobutamine,15767,2,0.0,14247


time: 1.93 s


In [9]:
big_df[big_df['label']=='vent_recieved']['value'].value_counts() #
#df["B"] = df["A"].astype('category')

#was origionally:
# None      8387
# Oxygen    3986
# Mech      2295

# now is:
# Mech      7144
# Oxygen    6176
# None       748

Mech      7144
Oxygen    6176
None       748
Name: value, dtype: int64

time: 608 ms


In [10]:
big_df[big_df['label']=='vent_recieved']['icustay_id'].nunique()
big_df.head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id
0,200012.0,1,daily_sofa_score,daily_sofa,0 days 00:00:00.000000000,2153-12-23,sofa,28448
1,200012.0,0.0,y/n,dobutamine,0 days 00:00:00.000000000,2153-12-23,dobutamine,28448
2,200012.0,0.0,y/n,dopamine,0 days 00:00:00.000000000,2153-12-23,dopamine,28448
3,200012.0,0.0,y/n,epinephrine,0 days 00:00:00.000000000,2153-12-23,epinephrine,28448
4,200012.0,asian,NaN,ethnicity,0 days 00:00:00.000000000,2153-12-23,pt_info,28448


time: 506 ms


In [11]:
big_df[big_df['label']=='vent_recieved'].head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id
13,200012.0,Oxygen,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2153-12-23,ventcategory,28448
109,200014.0,Mech,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2105-02-17,ventcategory,9514
500,200019.0,Mech,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2178-07-09,ventcategory,21789
850,200030.0,Oxygen,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2150-11-14,ventcategory,14987
1482,200033.0,Mech,mech/O2/none category,vent_recieved,0 days 00:00:00.000000000,2198-08-11,ventcategory,56369


time: 540 ms


In [12]:
# #converting venttype to category
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')
# #big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 906 µs


In [57]:
#converting categories to integers
vent_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])[1]
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']=pd.factorize(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])[0]#= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')

gender_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='gender','value'])[1]
big_df.loc[big_df.loc[:,'label']=='gender','value']=pd.factorize(big_df.loc[big_df.loc[:,'label']=='gender','value'])[0] #male=1

leukocyte_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[1]
big_df.loc[big_df.loc[:,'label']=='leukocyte','value']=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[0] 

nitrite_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[1]
big_df.loc[big_df.loc[:,'label']=='nitrite','value']=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[0] 


time: 6.68 s


## calc median/iqr for standardization

In [68]:
continuous=['daily_sofa',
            'first_admit_age',
            'pao2/fio2',
            'weight',
            'lactate',
            'mingcs',
            'diasbp',
            'heartrate',
            'meanartpress',
            'resprate',
            'spo2',
            'sysbp',
            'temperature',
            'bands',
            'hemoglobin',
            'platelet',
            'wbc',
            'height',
            'calcium',
            'glucose',
            'pco2',
            'ph',
            'bicarbonate',
            'bun',
            'chloride',
            'creatinine',
            'inr',
            'potassium',
            'ptt',
            'sodium',
            'bilirubin']

ordinal=[
            'leukocyte',
            'nitrite',
            'vent_recieved',
            'o2_flow',
            'rrt', 
            'phenylephrine',
            'norepinephrine',
            'vasopressin',
            'dobutamine',
            'dopamine',
            'epinephrine'
]
categorical=[
            "ethnicity",
            'gender'
]

time: 14.8 ms


In [69]:
final_bin=final_pt_df2[['icustay_id',"final_bin"]].copy()
#list of healthy pt
healthy_pt=list(final_bin[final_bin['final_bin']=="C_neg/A_partial"]['icustay_id'])
del(final_bin)

time: 5.62 ms


In [70]:
#formatting big_df to be limited to only numeric columns
#norm_df=big_df[big_df['label'].isin(label_list)].copy() #limiting to numerical variables
norm_df=big_df.copy()

categorical_dropped= ['ethnicity'] #dropping ethnicity from model
norm_df=norm_df[norm_df['label'].isin(continuous)]

norm_df['value']= pd.to_numeric(norm_df['value']) #converting to numeric 

#healthy patients filter
healthy_df=norm_df[norm_df['icustay_id'].isin(healthy_pt)].copy() #only numerical values for cneg/ab partial pt

time: 6 s


In [71]:
#finding the last 24 hours of each healthy patient.
healthy_pt_end=pd.DataFrame(healthy_df.groupby("icustay_id")['delta'].max())

healthy_pt_end["start"]=healthy_pt_end['delta']- pd.to_timedelta("1 day 00:00:00")
healthy_pt_end=healthy_pt_end.rename(columns={"delta":"end"}).reset_index()

healthy_df= pd.merge(healthy_df,healthy_pt_end, left_on="icustay_id", right_on="icustay_id", how="left" ) #now have the last 24 hours annotated for each patient as start and end.
healthy_df.head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id,end,start
0,200012.0,1.00,daily_sofa_score,daily_sofa,0 days 00:00:00.000000000,2153-12-23,sofa,28448,0 days 15:00:00.000000000,-1 days +15:00:00
1,200012.0,32.99,years,first_admit_age,0 days 00:00:00.000000000,2153-12-23,pt_info,28448,0 days 15:00:00.000000000,-1 days +15:00:00
2,200012.0,476.00,PaO2/FIO2 ratio,pao2/fio2,0 days 00:00:00.000000000,2153-12-23,pafaRatio,28448,0 days 15:00:00.000000000,-1 days +15:00:00
3,200012.0,51.20,kg,weight,0 days 00:00:00.000000000,2153-12-23,weight,28448,0 days 15:00:00.000000000,-1 days +15:00:00
4,200012.0,1.30,mmol/L,lactate,0 days 02:36:00.000000000,2153-12-23,labs,28448,0 days 15:00:00.000000000,-1 days +15:00:00


time: 1.71 s


In [72]:
#calculating medians and iqr for each label based on healthy patient's last 24 hours in icu
median_label=pd.DataFrame(healthy_df.groupby("label")['value'].median()).reset_index()
median_label=median_label.rename(columns={'value':"median"})
iqr_label=pd.DataFrame(healthy_df.groupby("label")['value'].quantile(0.75)-healthy_df.groupby("label")['value'].quantile(0.25)).reset_index()
iqr_label=iqr_label.rename(columns={'value':"iqr"})
median_label=pd.merge(median_label,iqr_label)  #final median df
del(iqr_label)
        
#saving the median label for standardization, will be used later on in teh notebook
pd.DataFrame(median_label).to_csv(Path(
    wd+'/data/processed/merged/%s_healthy_medians.csv' %(date)),index=False)

time: 1.62 s


In [18]:
# #making a test df of one patient so i can mess around with it
# test_df=norm_df[norm_df['icustay_id']==200095.0].copy() #299995 

# #merging in medians/iqr
# test_df=pd.merge(test_df, median_label)
# test_df.head()
# test_df['standardize']=((test_df['value']-test_df['median'])/test_df['iqr']).fillna(0)
# #normalizing

time: 1.24 ms


# aggregations

### dataformatting: convert all dtypes to a numeric type that pereserves nan. 

In [82]:
#splitting categorical, ordinal and continuous
big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy()

#continuous and ordinal variables
big_noCat= big_df.loc[big_df.loc[:,'label'].isin(continuous+ordinal),:].copy()
big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 
#changing pao2/fio2 ratio label
big_noCat.loc[big_noCat['label']=="pao2/fio2",'label']="pao2fio2Ratio"

time: 2min 48s


### adding a standardized value (x-median)/iqr  where median is of the last 24 hours in time window for culture neg/ ab partial patients 

In [83]:
big_noCat=pd.merge(big_noCat, median_label, how="left")
big_noCat['standardize']=((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).fillna(0)
big_noCat.loc[big_noCat['label'].isin(ordinal),'standardize']=big_noCat.loc[big_noCat['label'].isin(ordinal),'value'] #making all ordinal values in standardize equal to unstandardized
big_noCat.head()

,icustay_id,value,uom,label,delta,t_0,source,subject_id,median,iqr,standardize
0,200012.0,1.00,daily_sofa_score,daily_sofa,0 days 00:00:00.000000000,2153-12-23,sofa,28448,3.0,3.000,-0.666667
1,200012.0,0.00,y/n,dobutamine,0 days 00:00:00.000000000,2153-12-23,dobutamine,28448,NaN,NaN,0.000000
2,200012.0,0.00,y/n,dopamine,0 days 00:00:00.000000000,2153-12-23,dopamine,28448,NaN,NaN,0.000000
3,200012.0,0.00,y/n,epinephrine,0 days 00:00:00.000000000,2153-12-23,epinephrine,28448,NaN,NaN,0.000000
4,200012.0,32.99,years,first_admit_age,0 days 00:00:00.000000000,2153-12-23,pt_info,28448,65.2,26.065,-1.235757


time: 1.41 s


### min/max

In [116]:
hi_value= ['bands',
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
'pco2',
'potassium',
'ptt',
'resprate',
'temperature',
'weight', 'rrt', 'phenylephrine', 'norepinephrine', 'vasopressin', 'dobutamine', 'dopamine', 'epinephrine',
           'first_admit_age','leukocyte','nitrite','vent_recieved','o2_flow' ]

low_value=['bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp',
'pao2fio2Ratio']

both_value=['calcium',
'sodium',
'wbc']


time: 7.72 ms


In [117]:
#lambda x: x if not np.isnan(x) else -1

dict1={str(x):max for x in hi_value}
dict2={str(x):min for x in low_value } #dict2={str(x):min for x in low_value }
dict3={str(x):[max,min] for x in both_value }

time: 2.27 ms


In [118]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

dict1 = merge_two_dicts(dict1, dict2)
dict1 = merge_two_dicts(dict1, dict3)

time: 3.08 ms


In [119]:
dict1

{'bands': <function max>,
 'bicarbonate': <function min>,
 'bilirubin': <function max>,
 'bun': <function max>,
 'calcium': [<function max>, <function min>],
 'chloride': <function max>,
 'creatinine': <function max>,
 'daily_sofa': <function max>,
 'diasbp': <function min>,
 'dobutamine': <function max>,
 'dopamine': <function max>,
 'epinephrine': <function max>,
 'first_admit_age': <function max>,
 'glucose': <function max>,
 'heartrate': <function max>,
 'hemoglobin': <function min>,
 'inr': <function max>,
 'lactate': <function max>,
 'leukocyte': <function max>,
 'meanartpress': <function min>,
 'mingcs': <function min>,
 'nitrite': <function max>,
 'norepinephrine': <function max>,
 'o2_flow': <function max>,
 'pao2fio2Ratio': <function min>,
 'pco2': <function max>,
 'ph': <function min>,
 'phenylephrine': <function max>,
 'platelet': <function min>,
 'potassium': <function max>,
 'ptt': <function max>,
 'resprate': <function max>,
 'rrt': <function max>,
 'sodium': [<function 

time: 2.61 ms


# running all of the min/max aggregations.



In [120]:
##choose here if wanna use standardization or raw values.
values="standardize"

time: 783 µs


In [121]:
big_noCat.loc[big_noCat['label']=="leukocyte",:]

,icustay_id,value,uom,label,delta,t_0,source,subject_id,median,iqr,standardize
85,200012.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2153-12-23,leuk,28448,NaN,NaN,0.0
355,200014.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2105-02-17,leuk,9514,NaN,NaN,0.0
659,200019.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2178-07-09,leuk,21789,NaN,NaN,0.0
1060,200030.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2150-11-14,leuk,14987,NaN,NaN,0.0
1669,200033.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2198-08-11,leuk,56369,NaN,NaN,0.0
2240,200036.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2181-08-27,leuk,9960,NaN,NaN,0.0
2601,200055.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2179-03-17,leuk,25225,NaN,NaN,0.0
3069,200059.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2198-02-10,leuk,28494,NaN,NaN,0.0
4109,200061.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2134-01-24,leuk,74282,NaN,NaN,0.0
4479,200063.0,0.0,pos/neg category,leukocyte,1 days 00:00:00.000000000,2141-03-12,leuk,99052,NaN,NaN,0.0


time: 469 ms


In [122]:
big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
table = pd.pivot_table(big_max, values=values, columns='label', index=['icustay_id'],aggfunc=max, dropna=False)

time: 1.11 s


In [123]:
big_max[big_max['label']=='leukocyte']['value'].describe() #good the categorical stuff worked

count    14478.000000
mean         0.151955
std          0.358989
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: value, dtype: float64

time: 251 ms


In [124]:
big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
table2 = pd.pivot_table(big_min, values=values, columns='label', index=['icustay_id'],aggfunc=min, dropna=False)

time: 1.1 s


In [125]:
big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
table3 = pd.pivot_table(big_both, values=values, columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)

time: 410 ms


In [126]:
print(len(table),len(table2),len(table3)) #14668 14668 14618

14478 14478 14417
time: 1.09 ms


In [127]:
#first left join all different aggregations together. 
worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')
worst_df.head()

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,icustay_id,bands,bilirubin,bun,chloride,creatinine,daily_sofa,dobutamine,dopamine,epinephrine,...,ph,platelet,spo2,sysbp,"(max, calcium)","(max, sodium)","(max, wbc)","(min, calcium)","(min, sodium)","(min, wbc)"
0,200012.0,0.875,NaN,NaN,NaN,NaN,-0.666667,0.0,0.0,0.0,...,NaN,-0.450382,-0.50,-0.766667,NaN,NaN,-0.790323,NaN,NaN,-0.790323
1,200014.0,NaN,NaN,0.190476,1.125,-0.222222,-0.333333,0.0,0.0,0.0,...,-0.222222,-0.572519,-0.50,-1.600000,0.909091,0.8,0.725806,-0.090909,-0.2,0.145161
2,200019.0,NaN,NaN,0.190476,0.500,0.000000,1.000000,0.0,0.0,0.0,...,-0.111111,-0.396947,-8.75,-1.633333,NaN,0.2,0.596774,NaN,0.2,0.483871
3,200030.0,NaN,8.5625,0.380952,0.750,0.000000,0.666667,1.0,0.0,0.0,...,-0.111111,-0.229008,-1.25,-0.366667,1.727273,0.4,0.661290,0.090909,-0.2,0.435484
4,200033.0,NaN,NaN,-0.190476,0.375,-0.333333,-0.666667,0.0,0.0,0.0,...,0.888889,-0.595420,-0.25,-0.633333,-0.727273,1.2,-0.338710,-1.000000,-1.2,-0.580645


time: 81.3 ms


In [110]:
### formatting categorical to wide format to match the tables/worst_df
big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy()
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per patient per time. 

time: 189 ms


In [128]:
def merge_cat_agg(num_df, cat_df, twoclass=False):
    """
    merges the categorical and aggregated dataframes together. if twoclass=True (default=False), returns just the truepos trueneg twoclass patients.
    """
    
    worst_df=pd.merge(num_df, cat_df, left_on='icustay_id', right_on='icustay_id',how='left')
#     worst_df['leukocyte']=worst_df['leukocyte'].astype("category")
#     worst_df['nitrite']=worst_df['nitrite'].astype("category")
#     worst_df['vent_recieved']=worst_df['vent_recieved'].astype("category")
    worst_df['ethnicity']=worst_df['ethnicity'].astype("category")
    worst_df['gender']=worst_df['gender'].astype("category")
    
    baselinePts= final_pt_df2.loc[final_pt_df2.loc[:,'final_bin'].isin(["C_pos/A_full","C_neg/A_partial"]),"icustay_id"]
    
    if twoclass==True:
        worst_df= worst_df.loc[worst_df.loc[:,'icustay_id'].isin(baselinePts),:]
    return(worst_df)

time: 10.1 ms


In [129]:
worst_df=merge_cat_agg(worst_df, big_categorical, twoclass=False) #using max/min aggregates 


time: 15.5 ms


In [132]:
pd.DataFrame(worst_df).to_csv(Path(
    wd+'/data/processed/merged/%s_worstdf_std.csv' %(date)),index=False)

time: 829 ms


## 4-11-19: range, mean and stdev aggregations (newagg)

In [133]:
## adding more aggregations 4-11-19:

all_variables= [
#'bands', #removed due to sparesness
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
#'pco2', #removed due to sparesness
'potassium',
'ptt',
'resprate',
'temperature',
#'weight', 
#'rrt', 
#'phenylephrine',
#'norepinephrine',
#'vasopressin',
#'dobutamine',
#'dopamine',
#'epinephrine',
#'first_admit_age',
'bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp',
'pao2fio2Ratio', #need to rename- done
'calcium',
'sodium',
'wbc']

remaining_variables=[
    "bands",
    "pco2",
    'weight', 
    'rrt', 
    'phenylephrine',
    'norepinephrine',
    'vasopressin',
    'dobutamine',
    'dopamine',
    'epinephrine',
    'first_admit_age'
]


time: 10.6 ms


np.ptp = difference btween min and max


In [134]:
##choose here if wanna use standardization or raw values.
values="standardize"

time: 617 µs


In [135]:
big_range= big_noCat.loc[big_noCat.loc[:,'label'].isin(all_variables),:]
table4 = pd.pivot_table(big_range, values=values, columns='label', index=['icustay_id'],aggfunc=[np.ptp, np.mean, np.std], dropna=[False,False,False]) 
table4.columns=['_'.join(col).strip() for col in table4.columns.values]
table4=table4.reset_index()

time: 43.5 s


In [136]:
#grabing the rest of the variables not suitable for range or mean/std
remaining= big_noCat.loc[big_noCat.loc[:,'label'].isin(remaining_variables),:]
table5 = pd.pivot_table(remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 

time: 624 ms


In [137]:
#table5.columns
table5.columns = table5.columns.get_level_values(1)
table5=table5.reset_index()

time: 12.8 ms


In [138]:
table4.head()#.rename(columns={})

,icustay_id,ptp_bicarbonate,ptp_bilirubin,ptp_bun,ptp_calcium,ptp_chloride,ptp_creatinine,ptp_daily_sofa,ptp_diasbp,ptp_glucose,...,std_ph,std_platelet,std_potassium,std_ptt,std_resprate,std_sodium,std_spo2,std_sysbp,std_temperature,std_wbc
0,200012.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.277778,NaN,...,NaN,NaN,NaN,NaN,0.503421,NaN,0.355157,0.129624,0.337811,NaN
1,200014.0,1.0,NaN,0.476190,1.000000,0.875,0.222222,0.000000,2.777778,0.250000,...,0.299544,0.085800,0.164957,0.076444,1.000079,0.529150,0.349257,0.862796,0.706647,0.335236
2,200019.0,0.0,NaN,0.000000,NaN,0.000,0.111111,0.333333,3.833333,0.803571,...,NaN,0.043182,0.202031,NaN,0.871901,0.000000,1.886965,1.038963,0.435482,0.079835
3,200030.0,0.4,0.0,0.333333,1.636364,0.125,0.222222,1.666667,2.611111,3.571429,...,0.189758,0.076463,0.884485,0.131076,0.542263,0.251661,0.326499,0.442354,0.599855,0.116308
4,200033.0,1.2,NaN,0.285714,0.272727,1.000,0.333333,0.333333,2.388889,5.250000,...,0.129630,0.110884,0.321753,0.089977,0.754394,0.783156,0.279803,0.444738,0.730230,0.123188


time: 36.4 ms


In [139]:
list(table4)

['icustay_id',
 'ptp_bicarbonate',
 'ptp_bilirubin',
 'ptp_bun',
 'ptp_calcium',
 'ptp_chloride',
 'ptp_creatinine',
 'ptp_daily_sofa',
 'ptp_diasbp',
 'ptp_glucose',
 'ptp_heartrate',
 'ptp_hemoglobin',
 'ptp_inr',
 'ptp_lactate',
 'ptp_meanartpress',
 'ptp_mingcs',
 'ptp_pao2fio2Ratio',
 'ptp_ph',
 'ptp_platelet',
 'ptp_potassium',
 'ptp_ptt',
 'ptp_resprate',
 'ptp_sodium',
 'ptp_spo2',
 'ptp_sysbp',
 'ptp_temperature',
 'ptp_wbc',
 'mean_bicarbonate',
 'mean_bilirubin',
 'mean_bun',
 'mean_calcium',
 'mean_chloride',
 'mean_creatinine',
 'mean_daily_sofa',
 'mean_diasbp',
 'mean_glucose',
 'mean_heartrate',
 'mean_hemoglobin',
 'mean_inr',
 'mean_lactate',
 'mean_meanartpress',
 'mean_mingcs',
 'mean_pao2fio2Ratio',
 'mean_ph',
 'mean_platelet',
 'mean_potassium',
 'mean_ptt',
 'mean_resprate',
 'mean_sodium',
 'mean_spo2',
 'mean_sysbp',
 'mean_temperature',
 'mean_wbc',
 'std_bicarbonate',
 'std_bilirubin',
 'std_bun',
 'std_calcium',
 'std_chloride',
 'std_creatinine',
 'std_dai

time: 4.08 ms


In [140]:
## merging aggregations together
#first left join all different newagg together. 
table4=pd.merge(table4, table5, how='left')

time: 32.9 ms


In [141]:
### formatting categorical to wide format to match the aggregated numerical
big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy()
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per patient per time. 

time: 201 ms


### merging categorical with aggregated

In [142]:
#using fxn written for worst_df to make newagg_df
newagg_df=merge_cat_agg(table4, big_categorical, twoclass=False) #using range/mean/stdev aggregates 


time: 35.8 ms


In [144]:
newagg_df

,icustay_id,ptp_bicarbonate,ptp_bilirubin,ptp_bun,ptp_calcium,ptp_chloride,ptp_creatinine,ptp_daily_sofa,ptp_diasbp,ptp_glucose,...,epinephrine,first_admit_age,norepinephrine,pco2,phenylephrine,rrt,vasopressin,weight,ethnicity,gender
0,200012.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.277778,NaN,...,0.0,32.99,0.0,NaN,0.0,0.0,0.0,51.20,asian,0
1,200014.0,1.0,NaN,0.476190,1.000000,0.875,0.222222,0.000000,2.777778,0.250000,...,0.0,84.73,0.0,45.0,0.0,0.0,0.0,62.00,unknown/other,1
2,200019.0,0.0,NaN,0.000000,NaN,0.000,0.111111,0.333333,3.833333,0.803571,...,0.0,82.88,0.0,36.0,0.0,0.0,0.0,65.00,white/nonhispanic,0
3,200030.0,0.4,0.0000,0.333333,1.636364,0.125,0.222222,1.666667,2.611111,3.571429,...,0.0,54.19,0.0,34.0,0.0,0.0,0.0,113.60,black,1
4,200033.0,1.2,NaN,0.285714,0.272727,1.000,0.333333,0.333333,2.388889,5.250000,...,0.0,67.14,0.0,NaN,1.0,0.0,0.0,74.00,white/nonhispanic,1
5,200036.0,0.0,NaN,0.333333,NaN,0.375,0.111111,0.333333,3.111111,5.803571,...,0.0,74.93,0.0,NaN,0.0,0.0,0.0,79.00,white/nonhispanic,1
6,200055.0,0.2,0.0000,0.523810,0.000000,0.375,1.666667,2.333333,1.555556,2.071429,...,0.0,23.73,0.0,NaN,0.0,1.0,0.0,56.00,black,0
7,200059.0,2.0,0.0000,1.238095,1.454545,1.500,0.777778,1.000000,3.888889,1.553571,...,0.0,78.90,1.0,41.0,1.0,0.0,0.0,54.00,white/nonhispanic,0
8,200061.0,0.4,2.1250,0.142857,NaN,0.500,0.444444,0.333333,1.666667,3.803571,...,0.0,45.75,0.0,NaN,0.0,0.0,0.0,250.00,unknown/other,1
9,200063.0,0.4,0.0000,1.904762,1.818182,1.250,1.000000,0.000000,2.833333,1.250000,...,0.0,37.07,0.0,NaN,0.0,0.0,0.0,NaN,unknown/other,1


time: 63 ms


In [143]:
list(newagg_df)

['icustay_id',
 'ptp_bicarbonate',
 'ptp_bilirubin',
 'ptp_bun',
 'ptp_calcium',
 'ptp_chloride',
 'ptp_creatinine',
 'ptp_daily_sofa',
 'ptp_diasbp',
 'ptp_glucose',
 'ptp_heartrate',
 'ptp_hemoglobin',
 'ptp_inr',
 'ptp_lactate',
 'ptp_meanartpress',
 'ptp_mingcs',
 'ptp_pao2fio2Ratio',
 'ptp_ph',
 'ptp_platelet',
 'ptp_potassium',
 'ptp_ptt',
 'ptp_resprate',
 'ptp_sodium',
 'ptp_spo2',
 'ptp_sysbp',
 'ptp_temperature',
 'ptp_wbc',
 'mean_bicarbonate',
 'mean_bilirubin',
 'mean_bun',
 'mean_calcium',
 'mean_chloride',
 'mean_creatinine',
 'mean_daily_sofa',
 'mean_diasbp',
 'mean_glucose',
 'mean_heartrate',
 'mean_hemoglobin',
 'mean_inr',
 'mean_lactate',
 'mean_meanartpress',
 'mean_mingcs',
 'mean_pao2fio2Ratio',
 'mean_ph',
 'mean_platelet',
 'mean_potassium',
 'mean_ptt',
 'mean_resprate',
 'mean_sodium',
 'mean_spo2',
 'mean_sysbp',
 'mean_temperature',
 'mean_wbc',
 'std_bicarbonate',
 'std_bilirubin',
 'std_bun',
 'std_calcium',
 'std_chloride',
 'std_creatinine',
 'std_dai

time: 11.4 ms


In [145]:
pd.DataFrame(newagg_df).to_csv(Path(
    wd+'/data/processed/merged/%s_newaggdf_std.csv' %(date)),index=False)

time: 2.05 s


In [29]:
len(worst_df2)

7778

time: 2.63 ms


In [30]:
# big_categorical["icustay_id"].isin(icustay_id)

time: 680 µs


In [31]:
#big_noCat['value']=big_noCat['value'].astype(float, copy=False)
#big_noCat.groupby(['icustay_id','label'], as_index=False)['value'].agg(dict1) #

time: 635 µs


In [32]:
# test= big_noCat.groupby(['icustay_id'], as_index=False)['value'].agg(dict1)
# table = pd.pivot_table(big_noCat, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
# table

# pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
#                aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

time: 879 µs


In [33]:
# #table = pivot_table(df, values=['D', 'E'], index=['A', 'C'],aggfunc={})
# ##don't know why this doesn't work. 
# table = pd.pivot_table(big_noCat, values='value',columns='label', index='icustay_id',aggfunc=dict1, dropna=False)
# table


time: 1.15 ms


In [34]:
# big_agg= big_df.loc[big_df.loc[:,'label'].isin(non_category),:].groupby(['icustay_id','label'], as_index=False)['value'].apply(lambda x: float(x))

    #.agg({str(x):['max'] for x in hi_value})
#big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 713 µs


In [35]:
# hi_df= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]
# hi_df['value']=hi_df['value'].astype(float, copy=False)
# hi_df.groupby(['icustay_id','label'], as_index=False)['value'].agg('max') #this works

time: 801 µs


In [36]:
# %whos

# del big_agg
# del hi_df
# del final_pt_df2

time: 673 µs
